In [2]:
!pip install geopy

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import difflib

from geopy.distance import vincenty

In [30]:
## Import Data
# -----------------------------
# 1) Yelp Data Chicago
Yelp_df = pd.read_csv("../Data/Yelp_Restaurants_Chicago.csv")
# 2) Business License Chicago
License_df = pd.read_csv("../Data/Clean_Chicago_Restaurants_Licenses.csv")
len(License_df)

140454

In [6]:
License_Clean_df = License_df.groupby("DOING BUSINESS AS NAME").first().reset_index()
len(License_Clean_df)

28604

In [7]:
print(License_Clean_df.columns)
print(Yelp_df.columns)

Index(['DOING BUSINESS AS NAME', 'Unnamed: 0', 'LEGAL NAME', 'LICENSE CODE',
       'CITY', 'WARD', 'ZIP CODE', 'DATE ISSUED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')
Index(['zip', 'city', 'state', 'name', 'price', 'rating', 'review_count',
       'type', 'latitude', 'longitude'],
      dtype='object')


In [8]:
# Initialize the DataFrame
Yelp_License_df = Yelp_df
Yelp_License_df["Min_Dist"] = np.nan
Yelp_License_df["Min_Name"] = np.nan

Yelp_License_df.head()

,zip,city,state,name,price,rating,review_count,type,latitude,longitude,Min_Dist,Min_Name
0,60601,Chicago,IL,Wildberry Pancakes and Cafe,$$,4.5,5862,American (New),41.884668,-87.622880,NaN,NaN
1,60603,Chicago,IL,The Gage,$$,4.0,2632,American (New),41.881048,-87.624533,NaN,NaN
2,60603,Chicago,IL,Cindy's,$$,4.0,1387,American (New),41.881689,-87.625006,NaN,NaN
3,60603,Chicago,IL,The Marq,$$,4.0,509,American (New),41.879619,-87.629925,NaN,NaN
4,60603,Chicago,IL,Cherry Circle Room,$$$,4.5,330,American (New),41.881664,-87.624969,NaN,NaN


In [27]:
dist_list = License_Clean_df.apply(lambda x: vincenty((x["LATITUDE"], x["LONGITUDE"]), (41.884668,-87.622880)).meters, axis = 1)
name_corr_list = License_Clean_df.apply(lambda x: difflib.SequenceMatcher(None, x["DOING BUSINESS AS NAME"], "Boka").ratio(), axis = 1)
dist_index = dist_list[dist_list <= 50].index
filtered_name_corr = name_corr_list[dist_index]
largest_name_corr = max(filtered_name_corr)
largest_name_index = filtered_name_corr[filtered_name_corr==largest_name_corr].index[0]
print(largest_name_index) 
print(filtered_name_corr)

/Users/haolu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  """Entry point for launching an IPython kernel.


25984
7017     0.000000
9649     0.000000
10452    0.000000
10459    0.000000
19477    0.000000
22481    0.000000
23541    0.074074
25984    0.111111
27955    0.068966
dtype: float64


9

In [34]:
for index, row in Yelp_License_df.iterrows():
    lat = row["latitude"]
    lng = row["longitude"]
    name = row["name"]
    dist_list = License_Clean_df.apply(lambda x: vincenty((x["LATITUDE"], x["LONGITUDE"]), (lat,lng)).meters, axis = 1)
    name_corr_list = License_Clean_df.apply(lambda x: difflib.SequenceMatcher(None, x["DOING BUSINESS AS NAME"].lower(), name.lower()).ratio(), axis = 1)
    dist_filter_index = dist_list[dist_list <= 50].index
    filtered_name_corr = name_corr_list[dist_filter_index]
    if len(filtered_name_corr) != 0:
        largest_name_corr = max(filtered_name_corr)
        if largest_name_corr >= 0.5:
            largest_name_index = filtered_name_corr[filtered_name_corr==largest_name_corr].index[0]
            select_dist = dist_list[largest_name_index]
            select_name = License_Clean_df.loc[largest_name_index,"DOING BUSINESS AS NAME"]
            Yelp_License_df.loc[index,"Select_Dist"] = select_dist
            Yelp_License_df.loc[index,"Select_Name"] = select_name
            Yelp_License_df.loc[index,"Date_Issued"] = License_Clean_df.loc[largest_name_index,"DATE ISSUED"]
            print(f"Restaurant name is {name}, the closest business is {select_name} with dist as {select_dist} with corr as {largest_name_corr}")
    
Yelp_License_df.to_csv("Yelp_License_Merge.csv",index=False)

/Users/haolu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  """


Restaurant name is Wildberry Pancakes and Cafe, the closest business is WILDBERRY PANCAKES & CAFE with dist as 33.76170289689661 with corr as 0.9230769230769231
Restaurant name is The Marq, the closest business is THE MARQ with dist as 12.41688124910465 with corr as 1.0
Restaurant name is Cherry Circle Room, the closest business is THE CHERRY CIRCLE ROOM with dist as 41.58718178800911 with corr as 0.9
Restaurant name is The Berghoff Restaurant, the closest business is BERGHOFF'S RESTAURANT with dist as 21.276030923646445 with corr as 0.8636363636363636
Restaurant name is Girl & the Goat, the closest business is GIRL & THE GOAT with dist as 22.328920596394592 with corr as 1.0
Restaurant name is The Publican, the closest business is THE PUBLICAN with dist as 11.969668683050111 with corr as 1.0
Restaurant name is Roister, the closest business is ROISTER with dist as 12.300012137895708 with corr as 1.0
Restaurant name is Dusek's Board & Beer, the closest business is DUSEK'S BOARD AND BEER 

Restaurant name is Bohica Bar & Grill, the closest business is BOHICA BAR & GRILL with dist as 23.23088361447875 with corr as 1.0
Restaurant name is Tweet, the closest business is TWEET with dist as 4.622612817176248 with corr as 1.0
Restaurant name is Vincent Restaurant, the closest business is VINCENT with dist as 18.707311804609898 with corr as 0.56
Restaurant name is Band of Bohemia, the closest business is BAND OF BOHEMIA with dist as 36.89915056185533 with corr as 1.0
Restaurant name is Lady Gregory's, the closest business is LADY GREGORY'S with dist as 41.149738860922916 with corr as 1.0
Restaurant name is Fat Cat, the closest business is FAT CAT with dist as 10.5275392077365 with corr as 1.0
Restaurant name is Community Tavern, the closest business is COMMUNITY with dist as 17.771557313192602 with corr as 0.72
Restaurant name is Old Irving Brewing, the closest business is OLD IRVING BREWING with dist as 21.53989911846122 with corr as 1.0
Restaurant name is Horse Thief Hollow, t

Restaurant name is Forastero, the closest business is FORASTERO, LLC with dist as 20.808995704347726 with corr as 0.782608695652174
Restaurant name is P.F. Chang's, the closest business is P.F. CHANG CHINA BISTRO with dist as 22.855969676855413 with corr as 0.6285714285714286
Restaurant name is Earth's Kitchen, the closest business is EARTH'S KITCHEN with dist as 2.747110754117198 with corr as 1.0
Restaurant name is Del Seoul, the closest business is DEL SEOUL with dist as 7.569622723347946 with corr as 1.0
Restaurant name is Macku Sushi, the closest business is MACKU with dist as 15.957780371178558 with corr as 0.625
Restaurant name is Shine Restaurant, the closest business is SHINE RESTAURANT with dist as 2.579004590727237 with corr as 1.0
Restaurant name is Asian Cajun, the closest business is ASIAN CAJUN with dist as 17.430426618712985 with corr as 1.0
Restaurant name is Circle Sushi & Grill, the closest business is CIRCLE SUSHI & GRILL with dist as 18.91872642338842 with corr as 1

Restaurant name is Brasil Legal Cafe, the closest business is BRASIL LEGAL with dist as 16.913259398763934 with corr as 0.8275862068965517
Restaurant name is Norman's Bistro, the closest business is NORMAN'S with dist as 29.041743844208206 with corr as 0.6956521739130435
Restaurant name is Fogo de Chão Brazilian Steakhouse, the closest business is FOGO DE CHAO CHURRASCARIA (CHICAGO) LLC with dist as 18.744789260747105 with corr as 0.5277777777777778
Restaurant name is ZED451, the closest business is ZED 451 with dist as 40.721591514904475 with corr as 0.9230769230769231
Restaurant name is Brazilian Bowl, the closest business is The Brazilian Bowl with dist as 9.289538327346685 with corr as 0.875
Restaurant name is Elephant & Castle, the closest business is ELEPHANT & CASTLE with dist as 11.419977331288493 with corr as 1.0
Restaurant name is Elephant & Castle, the closest business is ELEPHANT & CASTLE PUB RESTAURANT with dist as 1.0916895727640967 with corr as 0.6938775510204082
Restaur

...

Restaurant name is Big Star, the closest business is BIG STAR with dist as 14.718614970140928 with corr as 1.0
Restaurant name is Antique Taco, the closest business is ANTIQUE TACO with dist as 15.474332556168637 with corr as 1.0
Restaurant name is Takito Kitchen, the closest business is TAKITO KITCHEN with dist as 20.128089731126813 with corr as 1.0
Restaurant name is Las Palmas, the closest business is LAS PALMAS with dist as 18.1826432216488 with corr as 1.0
Restaurant name is Sabroso! Mexican Grill, the closest business is SABROSO! MEXICAN GRILL with dist as 18.208836291931863 with corr as 1.0
Restaurant name is Taqueria Traspasada, the closest business is TRASPASADA #2 with dist as 14.738847597194573 with corr as 0.625
Restaurant name is Alegrias Seafood Chicago, the closest business is ALEGRIAS SEAFOOD with dist as 16.608396938045114 with corr as 0.8
Restaurant name is El Barco, the closest business is EL BARCO RESTAURANT with dist as 19.946245841673203 with corr as 0.59259259259

Restaurant name is Primo Chuki's, the closest business is TAQUERIA PRIMO CHUKIS UPTOWN with dist as 2.0583355786010435 with corr as 0.5853658536585366
Restaurant name is Carmela's Taqueria, the closest business is CARMELITA'S TAQUERIA with dist as 5.839899635448513 with corr as 0.9473684210526315
Restaurant name is Octavio Cantina & Kitchen, the closest business is OCTAVIO CANTINA & KITCHEN with dist as 11.998237029904473 with corr as 1.0
Restaurant name is El Taco Feliz Taqueria, the closest business is EL TACO FELIZ TAQUERIA with dist as 18.297503770497816 with corr as 1.0
Restaurant name is Sol de Mexico, the closest business is SOL DE MEXICO with dist as 25.842521988017392 with corr as 1.0
Restaurant name is La Oaxaqueña, the closest business is TAQUERIA LA OAXAQUENA with dist as 1.0544880309956146 with corr as 0.6666666666666666
Restaurant name is Mariscos La Costa, the closest business is LA COSTA with dist as 17.183525959793645 with corr as 0.64
Restaurant name is Uncle Julio's,

Restaurant name is Tasty Polish Food, the closest business is POLISH FOOD with dist as 18.914025266924785 with corr as 0.7857142857142857
Restaurant name is Polish House Polska Chata, the closest business is POLISH HOUSE. LUNCH, DINNER & SPECIALS LLC with dist as 13.988600501232886 with corr as 0.5074626865671642
Restaurant name is Krokodile, the closest business is PINK KROKODILE CAFE with dist as 17.937804019716747 with corr as 0.6428571428571429
Restaurant name is Olivia's Taste, the closest business is Olivia's Taste Inc with dist as 41.98463046735812 with corr as 0.875
Restaurant name is Ferajna, the closest business is FERAJNA RESTAURANT with dist as 22.302725128928504 with corr as 0.56
Restaurant name is Retro Cafe and Restaurant, the closest business is RETRO CAFE RESTAURANT with dist as 17.019355696189667 with corr as 0.9130434782608695
Restaurant name is Karolinka Club, the closest business is KAROLINKA CLUB with dist as 16.885950928173386 with corr as 1.0
Restaurant name is 

Restaurant name is Green Leaf Thai Cuisine, the closest business is GREEN LEAF THAI CUISINE with dist as 8.759102266468044 with corr as 1.0
Restaurant name is Siam Country, the closest business is SIAM COUNTRY with dist as 2.7193018556817674 with corr as 1.0
Restaurant name is Rice Thai Cafe, the closest business is RICE THAI CAFE with dist as 15.698926411643395 with corr as 1.0
Restaurant name is Ruk Sushi & Thai, the closest business is RUK SUSHI AND THAI with dist as 24.23417557038677 with corr as 0.8823529411764706
Restaurant name is Taste of Thai Town, the closest business is TASTE OF THAI TOWN with dist as 18.45973201044205 with corr as 1.0
Restaurant name is Noodles Party, the closest business is NOODLES PARTY with dist as 17.97832158590133 with corr as 1.0
Restaurant name is Siam Treasure Thai Restaurant, the closest business is SIAM TREASURE RESTAURANT with dist as 16.590737763264876 with corr as 0.9056603773584906
Restaurant name is Lotus Garden, the closest business is Lotus

Restaurant name is Nhu Lan Saigon Subs, the closest business is NHU LAN BAKERY with dist as 4.969611620348684 with corr as 0.5454545454545454
Restaurant name is LD Pho, the closest business is LD PHO with dist as 6.9479998389442486 with corr as 1.0
Restaurant name is LC Pho, the closest business is L C PHO RESTAURANT with dist as 16.961337860855338 with corr as 0.5
Restaurant name is 2 Asian Brothers, the closest business is 2 ASIAN BROTHERS with dist as 15.8961706831353 with corr as 1.0
Restaurant name is Far East Kitchen, the closest business is FAR EAST KITCHEN, LLC with dist as 6.831814369008354 with corr as 0.8648648648648649
Restaurant name is Lotus Garden, the closest business is Lotus Garden with dist as 19.959556815040024 with corr as 1.0
Restaurant name is Tank Noodle, the closest business is TANK NOODLE PHO XE TANG with dist as 7.889514923555746 with corr as 0.6470588235294118
Restaurant name is Ba Le Sandwich, the closest business is BALE SANDWICH AND BAKERY with dist as 25